## TRABAJO PRÁCTICO N° 1 - PROCESAMIENTO DEL LENGUAJE NATURAL - TUIA
Estudiantes:
* Menescaldi, Brisa
* Yañez, Mirian

Año: 2024

• Instalamos las liberías

In [54]:
!pip install requests beautifulsoup4 pandas matplotlib
!pip install transformers sentence_transformers
!pip install python-Levenshtein sentence-transformers
!pip install optuna
!pip install pyTelegramBotAPI

In [55]:
import requests
from bs4 import BeautifulSoup

* Obtenemos todos los generos de los libros del sitio mediante la técnica de web scraping


In [56]:
def preprocess_text(text):
    return text.lower()

data = requests.get('https://ww3.lectulandia.com/', verify=False)
soup = BeautifulSoup(data.text, 'lxml')
section = soup.find(id='secgenero')
generos = {}
if section:
    list_items = section.find_all('li')
    for li in list_items:
        link = li.find('a')
        if link:
            url = link.get('href')
            generos[preprocess_text(li.get_text())] = 'https://ww3.lectulandia.com' + url
elegidos = generos
for k,v in generos.items():
  print(k,v)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


arqueología https://ww3.lectulandia.com/genero/arqueologia/
arquitectura https://ww3.lectulandia.com/genero/arquitectura/
arte https://ww3.lectulandia.com/genero/arte/
astrología https://ww3.lectulandia.com/genero/astrologia/
astronomía https://ww3.lectulandia.com/genero/astronomia/
autoayuda https://ww3.lectulandia.com/genero/autoayuda/
autobiográfico https://ww3.lectulandia.com/genero/autobiografico/
aventuras https://ww3.lectulandia.com/genero/aventuras/
biografía https://ww3.lectulandia.com/genero/biografia/
biología https://ww3.lectulandia.com/genero/biologia/
bélico https://ww3.lectulandia.com/genero/belico/
ciencia https://ww3.lectulandia.com/genero/ciencia/
ciencia ficción https://ww3.lectulandia.com/genero/ciencia-ficcion/
ciencias exactas https://ww3.lectulandia.com/genero/ciencias-exactas/
ciencias naturales https://ww3.lectulandia.com/genero/ciencias-naturales/
ciencias sociales https://ww3.lectulandia.com/genero/ciencias-sociales/
cine https://ww3.lectulandia.com/genero/ci

* Armamos un dataset auxiliar para saber cuál es la similitud entre los generos, para ello vamos a tomar 3 libros de cada género

In [57]:
import re
import pandas as pd

libros = {'Genero': [], 'Sintesis': []}
for k in elegidos:
  data = requests.get(elegidos[k], verify=False)
  soup = BeautifulSoup(data.text, 'lxml')
  maximo = 0
  for i in soup.find_all("div", {"class":"details"}):
    link = 'https://ww3.lectulandia.com'+i.a['href']
    data_link = requests.get(link, verify=False)
    soup = BeautifulSoup(data_link.text, 'lxml')
    titulo = soup.find("div", {"id":"title"})
    autor = soup.find("div", {"id":"autor"}).text.split('Autor: ')[1]
    genero = soup.find("div", {"id":"genero"}).text.split('Generos: ')[1]
    sinopsis = soup.find("div", {"id":"sinopsis"})
    libros['Genero'].append(re.split(r'\s{2,}', genero))
    libros['Sintesis'].append(sinopsis.text.replace('\n', '').replace('\r', '').replace('\t', '').strip())
    if maximo > 3:
      break
    else:
      maximo += 1

df = pd.DataFrame(libros)
df.to_csv('generos.csv')
libros['Genero']

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connect

[['Arqueología', 'Divulgación'],
 ['Arqueología', 'Ciencias naturales', 'Divulgación'],
 ['Arqueología', 'Ciencias naturales', 'Divulgación', 'Ensayo'],
 ['Arqueología', 'Ciencias naturales', 'Divulgación', 'Ensayo'],
 ['Arqueología', 'Humor', 'Otros', 'Poesía', 'Referencia', 'Sátira'],
 ['Arquitectura', 'Divulgación', 'Geografía'],
 ['Arquitectura', 'Arte', 'Crónica', 'Ensayo', 'Memorias'],
 ['Arqueología',
  'Arquitectura',
  'Arte',
  'Ciencias sociales',
  'Historia',
  'Referencia'],
 ['Arquitectura', 'Historia', 'Referencia', 'Tecnología'],
 ['Arquitectura', 'Referencia'],
 ['Arte', 'Divulgación', 'Ensayo', 'Historia'],
 ['Arte', 'Ciencias sociales', 'Divulgación', 'Ensayo'],
 ['Arte', 'Divulgación'],
 ['Arte', 'Crítica y teoría literaria', 'Ensayo', 'Política'],
 ['Arte', 'Cinematografía', 'Ensayo'],
 ['Astrología', 'Magia', 'Referencia'],
 ['Astronomía', 'Divulgación'],
 ['Astronomía', 'Biología', 'Divulgación'],
 ['Astronomía', 'Divulgación', 'Otros'],
 ['Astronomía', 'Crónica

* Obtenemos los generos que menos relación tengan entre si

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import ast

nltk.download('stopwords')

data = pd.read_csv('generos.csv')

data['Genero'] = data['Genero'].apply(ast.literal_eval)

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]', ' ', text)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('spanish')])
    return text

data['Sintesis_procesada'] = data['Sintesis'].apply(preprocess_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Sintesis_procesada'])

cosine_sim = cosine_similarity(X)

category_similarities = {category: [] for categories in data['Genero'] for category in categories}

for idx, categories in enumerate(data['Genero']):
    for category in categories:
        similarity_scores = []
        for other_idx in range(len(data)):
            if idx != other_idx:
                similarity_scores.append(cosine_sim[idx][other_idx])
        if similarity_scores:
            average_similarity = sum(similarity_scores) / len(similarity_scores)
            category_similarities[category].append(average_similarity)

average_category_similarities = {category: sum(sim_scores) / len(sim_scores) for category, sim_scores in category_similarities.items() if sim_scores}

least_related_categories = sorted(average_category_similarities, key=average_category_similarities.get)[:11]

print("Las 11 categorías menos relacionadas son:", least_related_categories)
for last in least_related_categories:
  elegidos[last] = generos[last.lower()]
  print(elegidos[last], generos[last.lower()])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Las 11 categorías menos relacionadas son: ['Espionaje', 'Recetas de cocina', 'Diccionarios y enciclopedias', 'Ficción', 'Manuales y cursos', 'Novela del Oeste', 'Idiomas', 'Autobiográfico', 'Aventuras', 'Negocios', 'Novela Negra']
https://ww3.lectulandia.com/genero/espionaje/ https://ww3.lectulandia.com/genero/espionaje/
https://ww3.lectulandia.com/genero/recetas-de-cocina/ https://ww3.lectulandia.com/genero/recetas-de-cocina/
https://ww3.lectulandia.com/genero/diccionarios-y-enciclopedias/ https://ww3.lectulandia.com/genero/diccionarios-y-enciclopedias/
https://ww3.lectulandia.com/genero/ficcion/ https://ww3.lectulandia.com/genero/ficcion/
https://ww3.lectulandia.com/genero/manuales-y-cursos/ https://ww3.lectulandia.com/genero/manuales-y-cursos/
https://ww3.lectulandia.com/genero/novela-del-oeste/ https://ww3.lectulandia.com/genero/novela-del-oeste/
https://ww3.lectulandia.com/genero/idiomas/ https://ww3.lectulandia.com/genero/idiomas/
https://ww3.lectulandia.com/genero/autobiografico

* Almacenamos las URL's para cada género

In [59]:
elegidos = {
    'Ficción': 'https://ww3.lectulandia.com/genero/ficcion/',
    'Deporte': 'https://ww3.lectulandia.com/genero/deporte/',
    'Cómic': 'https://ww3.lectulandia.com/genero/comic/',
    'Crítica': 'https://ww3.lectulandia.com/genero/critica/',
    'Informática': 'https://ww3.lectulandia.com/genero/informatica/',
    'Hogar': 'https://ww3.lectulandia.com/genero/hogar/',
    'Policial': 'https://ww3.lectulandia.com/genero/policial/',
    'Espiritualidad': 'https://ww3.lectulandia.com/genero/espiritualidad/',
    'Gastronomía': 'https://ww3.lectulandia.com/genero/gastronomia/',
    'Viajes': 'https://ww3.lectulandia.com/genero/viajes/'
}

* Creamos un scraper para armar nuestro dataset

In [60]:
libros = {'Titulo': [], 'Autor': [], 'Genero': [], 'Sintesis': [], 'Url':[]}
for k in elegidos:
  data = requests.get(elegidos[k], verify=False)
  soup = BeautifulSoup(data.text, 'lxml')
  maximo = 0
  for i in soup.find_all("div", {"class":"details"}):
    link = 'https://ww3.lectulandia.com'+i.a['href']
    data_link = requests.get(link, verify=False)
    soup = BeautifulSoup(data_link.text, 'lxml')
    titulo = soup.find("div", {"id":"title"})
    autor = soup.find("div", {"id":"autor"}).text.split('Autor: ')[1]
    genero = soup.find("div", {"id":"genero"}).text.split('Generos: ')[1]
    sinopsis = soup.find("div", {"id":"sinopsis"})
    libros['Url'].append(link)
    libros['Titulo'].append(titulo.text.replace('\n', '').replace('\r', '').replace('\t', '').strip())
    libros['Autor'].append(autor.lower())
    libros['Genero'].append(k.lower())
    libros['Sintesis'].append(sinopsis.text.replace('\n', '').replace('\r', '').replace('\t', '').strip())
    if maximo > 20:
      break
    else:
      maximo += 1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ww3.lectulandia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connect

* Con los datos obtenidos armamos el dataset y lo exportamos a un CSV

In [61]:
df = pd.DataFrame(libros)
df.to_csv('libros.csv')

In [62]:
generos_unicos = df['Genero'].unique()
numero_de_generos = len(generos_unicos)
print(f"Número de géneros únicos: {numero_de_generos}")
titulos_unicos = df['Titulo'].unique()
numero_de_titulos = len(titulos_unicos)
print(f"Número de títulos únicos: {numero_de_titulos}")

Número de géneros únicos: 10
Número de títulos únicos: 117


* Procesamos el texto
* Implementamos un clasificador de género literario utilizando el modelo BERT para obtener embeddings de texto y un modelo de regresión logística para la clasificación.

In [63]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
import joblib

nltk.download('stopwords')
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[\W_]+', ' ', text)
    text = ' '.join(word for word in text.split() if word not in spanish_stop_words)
    return text

def preprocess_texts(texts):
    return [clean_text(text) for text in texts]

def get_bert_embeddings(texts, model):
    return np.array([model.encode(text) for text in texts])

def predecir_genero(modelo, label_to_genero, texto):
    texto_cleaned = preprocess_texts([texto])
    texto_vectorized = get_bert_embeddings(texto_cleaned, model)
    prediccion_label = modelo.predict(texto_vectorized)
    prediccion_genero = label_to_genero[prediccion_label[0]]
    return prediccion_genero

def clasificador(data):
    tipos_genero = data['Genero'].unique()
    genero_to_label = {genero: idx for idx, genero in enumerate(tipos_genero)}
    label_to_genero = {idx: genero for genero, idx in genero_to_label.items()}
    data['Genero_Label'] = data['Genero'].map(genero_to_label)
    X = data['Sintesis']
    y = data['Genero_Label']
    X_cleaned = preprocess_texts(X)
    X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2, random_state=42)
    X_train_vectorized = get_bert_embeddings(X_train, model)
    X_test_vectorized = get_bert_embeddings(X_test, model)
    modelo_LR = LogisticRegression(max_iter=10000, multi_class='multinomial', solver='lbfgs')
    modelo_LR.fit(X_train_vectorized, y_train)
    y_pred_LR = modelo_LR.predict(X_test_vectorized)
    acc_LR = accuracy_score(y_test, y_pred_LR)
    labels = np.unique(y_test)
    target_names = [label_to_genero[label] for label in labels]
    report_LR = classification_report(y_test, y_pred_LR, labels=labels, target_names=target_names, zero_division=1)
    print("Precisión Regresión Logística:", acc_LR)
    print("Reporte de clasificación Regresión Logística:\n", report_LR)
    return modelo_LR, label_to_genero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


* Obtenemos las métricas de ese modelo

In [64]:
modelo, generos = clasificador(df)

Precisión Regresión Logística: 0.5833333333333334
Reporte de clasificación Regresión Logística:
                 precision    recall  f1-score   support

       ficción       0.40      0.33      0.36         6
       deporte       1.00      1.00      1.00         1
       crítica       1.00      0.00      0.00         1
   informática       1.00      0.00      0.00         2
         hogar       0.50      1.00      0.67         3
espiritualidad       1.00      0.75      0.86         4
   gastronomía       1.00      0.00      0.00         2
        viajes       0.56      1.00      0.71         5

      accuracy                           0.58        24
     macro avg       0.81      0.51      0.45        24
  weighted avg       0.69      0.58      0.51        24



Debido a la baja precisión, vamos a probar con otros modelos de clasificación:

* Redes Neuronales
* Random Forest
* Gradient Boosting

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))

# Cargar el modelo
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[\W_]+', ' ', text)
    text = ' '.join(word for word in text.split() if word not in spanish_stop_words)
    return text

def preprocess_texts(texts):
    return [clean_text(text) for text in texts]

def get_bert_embeddings(texts, model):
    return np.array([model.encode(text) for text in texts])

def clasificador(data):
    tipos_genero = data['Genero'].unique()
    genero_to_label = {genero: idx for idx, genero in enumerate(tipos_genero)}
    label_to_genero = {idx: genero for genero, idx in genero_to_label.items()}
    data['Genero_Label'] = data['Genero'].map(genero_to_label)
    X = data['Sintesis']
    y = data['Genero_Label']
    X_cleaned = preprocess_texts(X)
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(pd.DataFrame(X_cleaned, columns=['Sintesis']), y)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled['Sintesis'], y_resampled, test_size=0.2, random_state=42)
    X_train_vectorized = get_bert_embeddings(X_train, model)
    X_test_vectorized = get_bert_embeddings(X_test, model)

    # Redes Neuronales
    mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
    mlp_model.fit(X_train_vectorized, y_train)
    y_pred_mlp = mlp_model.predict(X_test_vectorized)
    acc_mlp = accuracy_score(y_test, y_pred_mlp)
    print("Precisión Redes Neuronales:", acc_mlp)
    print("Reporte de clasificación Redes Neuronales:\n", classification_report(y_test, y_pred_mlp))

    # Random Forest
    rf_model = RandomForestClassifier(n_estimators=100)
    rf_model.fit(X_train_vectorized, y_train)
    y_pred_rf = rf_model.predict(X_test_vectorized)
    acc_rf = accuracy_score(y_test, y_pred_rf)
    print("Precisión Random Forest:", acc_rf)
    print("Reporte de clasificación Random Forest:\n", classification_report(y_test, y_pred_rf))

    # Gradient Boosting
    gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    gb_model.fit(X_train_vectorized, y_train)
    y_pred_gb = gb_model.predict(X_test_vectorized)
    acc_gb = accuracy_score(y_test, y_pred_gb)
    print("Precisión Gradient Boosting:", acc_gb)
    print("Reporte de clasificación Gradient Boosting:\n", classification_report(y_test, y_pred_gb))

    return mlp_model, rf_model, gb_model, label_to_genero
test_data = pd.read_csv('libros.csv')
modelo_mlp, modelo_rf, modelo_gb, generos = clasificador(test_data)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Precisión Redes Neuronales: 0.8863636363636364
Reporte de clasificación Redes Neuronales:
               precision    recall  f1-score   support

           0       0.43      0.75      0.55         4
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00         4
           9       1.00      0.56      0.71         9

    accuracy                           0.89        44
   macro avg       0.91      0.93      0.91        44
weighted avg       0.93      0.89      0.89        44

Precisión Random Forest: 0.75
Reporte de clasificación Random Forest:
               precision    recall  f1-score   support

           0       0.27 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


* Optamos por implementar el modelo de clasificación del perceptron multicapa para nuestro problema ya que es el que mejores resultados nos da. A continuación realizamos un ajuste de hiperparámetros para mejorar nuestras métricas

In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
import nltk
import optuna

nltk.download('stopwords')
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))

# Cargar el modelo
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[\W_]+', ' ', text)
    text = ' '.join(word for word in text.split() if word not in spanish_stop_words)
    return text

def preprocess_texts(texts):
    return [clean_text(text) for text in texts]

def get_bert_embeddings(texts, model):
    return np.array([model.encode(text) for text in texts])

def objective(trial, X_train, X_test, y_train, y_test):
    params = {
        'hidden_layer_sizes': trial.suggest_categorical('hidden_layer_sizes', [(20,), (50,), (100,), (150,),(200,)]),
        'max_iter': trial.suggest_int('max_iter', 500, 3000, step=100),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-2),
        'learning_rate': trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive']),
        'solver': trial.suggest_categorical('solver', ['adam', 'sgd', 'lbfgs']),
        'activation': trial.suggest_categorical('activation', ['relu', 'logistic', 'tanh', 'identity']),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'momentum': trial.suggest_uniform('momentum', 0.1, 0.9)
    }
    mlp_model = MLPClassifier(**params)
    mlp_model.fit(X_train, y_train)
    y_pred = mlp_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

def clasificador(data):
    tipos_genero = data['Genero'].unique()
    genero_to_label = {genero: idx for idx, genero in enumerate(tipos_genero)}
    label_to_genero = {idx: genero for genero, idx in genero_to_label.items()}
    data['Genero_Label'] = data['Genero'].map(genero_to_label)
    X = data['Sintesis']
    y = data['Genero_Label']
    X_cleaned = preprocess_texts(X)
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(pd.DataFrame(X_cleaned, columns=['Sintesis']), y)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled['Sintesis'], y_resampled, test_size=0.2, random_state=42)
    X_train_vectorized = get_bert_embeddings(X_train, model)
    X_test_vectorized = get_bert_embeddings(X_test, model)

    # Optimización de hiperparámetros con Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train_vectorized, X_test_vectorized, y_train, y_test), n_trials=10)
    best_params = study.best_params
    print("Mejores hiperparámetros:", best_params)

    # Redes Neuronales
    mlp_model = MLPClassifier(**best_params)
    mlp_model.fit(X_train_vectorized, y_train)
    y_pred_mlp = mlp_model.predict(X_test_vectorized)
    acc_mlp = accuracy_score(y_test, y_pred_mlp)
    print("Precisión Redes Neuronales:", acc_mlp)
    print("Reporte de clasificación Redes Neuronales:\n", classification_report(y_test, y_pred_mlp))

    return mlp_model, label_to_genero

test_data = pd.read_csv('libros.csv')
modelo_mlp, generos = clasificador(test_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[I 2024-06-07 01:08:20,539] A new study created in memory with name: no-name-4eb389ca-bd16-4b7e-87e5-821289d120ef
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (20,) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) whic

Mejores hiperparámetros: {'hidden_layer_sizes': (100,), 'max_iter': 2200, 'alpha': 2.4707039689018906e-05, 'learning_rate': 'invscaling', 'solver': 'lbfgs', 'activation': 'identity', 'batch_size': 128, 'momentum': 0.5046142529382656}
Precisión Redes Neuronales: 0.8863636363636364
Reporte de clasificación Redes Neuronales:
               precision    recall  f1-score   support

           0       0.57      1.00      0.73         4
           1       0.75      0.75      0.75         4
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00         4
           9       1.00      0.56      0.71         9

    accuracy                           0.89        44
   macro avg       0.90  

* Podemos observar que mediante la optimización con optuna mejoraron las métricas

* Realizamos algunos test para comparar como predicen los diferentes modelos

In [67]:
def preprocesar_nueva_sinopsis(sinopsis):
    sinopsis_limpia = clean_text(sinopsis)
    return get_bert_embeddings([sinopsis_limpia], model)

def predecir_genero_sinopsis(sinopsis, modelo):
    sinopsis_vectorizada = preprocesar_nueva_sinopsis(sinopsis)
    genero_predicho = modelo.predict(sinopsis_vectorizada)
    return generos[genero_predicho[0]]
print(test_data['Genero'].unique())

genero_predicho_mlp = predecir_genero_sinopsis("olimpicos", modelo_mlp)
genero_predicho_rf = predecir_genero_sinopsis("olimpicos", modelo_rf)
genero_predicho_gb = predecir_genero_sinopsis("olimpicos", modelo_gb)
print(f"Predicción Redes Neuronales: {genero_predicho_mlp}")
print(f"Predicción Random Forest: {genero_predicho_rf}")
print(f"Predicción Gradient Boosting: {genero_predicho_gb}")


['ficción' 'deporte' 'cómic' 'crítica' 'informática' 'hogar' 'policial'
 'espiritualidad' 'gastronomía' 'viajes']
Predicción Redes Neuronales: deporte
Predicción Random Forest: deporte
Predicción Gradient Boosting: deporte


In [79]:
genero_predicho_mlp = predecir_genero_sinopsis("comer una pizza", modelo_mlp)
genero_predicho_rf = predecir_genero_sinopsis("comer una pizza", modelo_rf)
genero_predicho_gb = predecir_genero_sinopsis("comer una pizza", modelo_gb)
print(f"Predicción Redes Neuronales: {genero_predicho_mlp}")
print(f"Predicción Random Forest: {genero_predicho_rf}")
print(f"Predicción Gradient Boosting: {genero_predicho_gb}")
print('-----------------------------')
genero_predicho_mlp = predecir_genero_sinopsis("futbol", modelo_mlp)
genero_predicho_rf = predecir_genero_sinopsis("futbol", modelo_rf)
genero_predicho_gb = predecir_genero_sinopsis("futbol", modelo_gb)
print(f"Predicción Redes Neuronales: {genero_predicho_mlp}")
print(f"Predicción Random Forest: {genero_predicho_rf}")
print(f"Predicción Gradient Boosting: {genero_predicho_gb}")
print('-----------------------------')
genero_predicho_mlp = predecir_genero_sinopsis("quiero aprender a programar", modelo_mlp)
genero_predicho_rf = predecir_genero_sinopsis("quiero aprender a programar", modelo_rf)
genero_predicho_gb = predecir_genero_sinopsis("quiero aprender a programar", modelo_gb)
print(f"Predicción Redes Neuronales: {genero_predicho_mlp}")
print(f"Predicción Random Forest: {genero_predicho_rf}")
print(f"Predicción Gradient Boosting: {genero_predicho_gb}")

Predicción Redes Neuronales: gastronomía
Predicción Random Forest: informática
Predicción Gradient Boosting: informática
-----------------------------
Predicción Redes Neuronales: deporte
Predicción Random Forest: deporte
Predicción Gradient Boosting: deporte
-----------------------------
Predicción Redes Neuronales: informática
Predicción Random Forest: informática
Predicción Gradient Boosting: informática


In [69]:
import Levenshtein

def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    return 1 - (distance / max_len)


* Implementamos las funciones de recomendación de libros que posteriormente serán utilizadas en los bot

In [70]:
from sentence_transformers import SentenceTransformer, util
lista_generos = [ v for k,v in generos.items()]
def resenas(url):
    resenas = []
    respuesta = requests.get(url)
    if respuesta.status_code == 200:
        html = BeautifulSoup(respuesta.text, 'html.parser')
        div = html.find('ol', attrs={'class': 'comment-list'})
        if div is None:
            resenas.append('No Tiene Reseña')
        else:
            ps = div.find_all('p')
            for p in ps:
                resenas.append(p.text)
    return resenas
def recomendacion_directa():
  tematica = input('Ingrese la temática deseada: ').lower()
  prediccion = predecir_genero_sinopsis(tematica, modelo_mlp)
  print(f"El género predicho es: {prediccion}")
  df_genero = df.loc[df['Genero'] == prediccion]
  pregunta = [tematica]
  modelo_genero = SentenceTransformer('msmarco-MiniLM-L-6-v3')
  incrustacion_consulta = modelo_genero.encode(pregunta)
  incrustaciones_respuestas = modelo_genero.encode(df_genero['Sintesis'].tolist())
  hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=3)
  print("Recomendados: ")
  for hit in hits[0]:
      indice_respuesta = hit['corpus_id']
      respuesta_similar = df_genero.iloc[indice_respuesta]
      print('Título: ', respuesta_similar['Titulo'])
      print('Autor: ', respuesta_similar['Autor'])
      print('Género: ', respuesta_similar['Genero'])
      print('Sintesis: ', respuesta_similar['Sintesis'])
      print('Reseña: ')
      list_resenas = resenas(respuesta_similar['Url'])
      for rs in list_resenas:
        print(rs)
      print('-----------------------------------')

modelo_autor = SentenceTransformer('msmarco-MiniLM-L-6-v3')

def recomendacion_autor():
    print('Autores disponibles: ')
    autores = df['Autor'].unique()
    autores_lista = []
    for autor in autores:
        autores_lista.append(autor.lower())
    for i in range(0, len(autores), 4):
        print(f'{autores[i]:<25}', end='')
        if i+1 < len(autores):
            print(f'{autores[i+1]:<25}', end='')
        if i+2 < len(autores):
            print(f'{autores[i+2]:<25}', end='')
        if i+3 < len(autores):
            print(f'{autores[i+3]:<25}')
    print()

    consulta_autor = input('Ingrese el autor que desea: ').lower()

    # Cargar el modelo de SentenceTransformer
    modelo_autor = SentenceTransformer('msmarco-MiniLM-L-6-v3')

    # Generar la incrustación para la consulta
    incrustacion_consulta = modelo_autor.encode([consulta_autor])
    incrustaciones_respuestas = modelo_autor.encode(autores_lista)
    hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=6)
    subcadenas = []
    for j, hit in enumerate(hits[0]):
        autor_similar = autores_lista[hit['corpus_id']]
        if consulta_autor in autor_similar:
            subcadenas.append(autor_similar)

    recomendaciones = []
    autores_considerados = set()

    def agregar_libros(autor, max_libros):
        libros_autor = df[df['Autor'].str.lower() == autor].head(max_libros)
        for _, libro in libros_autor.iterrows():
            recomendaciones.append(libro)
        return len(libros_autor)

    if not subcadenas:
        mejor_autor = autores_lista[hits[0][0]['corpus_id']]
        autores_considerados.add(mejor_autor)
        agregar_libros(mejor_autor, 3)
    else:
        for autor in subcadenas:
            if len(recomendaciones) >= 3:
                break
            autores_considerados.add(autor)
            libros_agregados = agregar_libros(autor, 3 - len(recomendaciones))

    if len(recomendaciones) < 3:
        for hit in hits[0]:
            autor_similar = autores_lista[hit['corpus_id']]
            if autor_similar not in autores_considerados:
                autores_considerados.add(autor_similar)
                libros_agregados = agregar_libros(autor_similar, 3 - len(recomendaciones))
                if len(recomendaciones) >= 3:
                    break

    for libro in recomendaciones:
        print('Título: ', libro['Titulo'])
        print('Autor: ', libro['Autor'])
        print('Género: ', libro['Genero'])
        print('Sintesis: ', libro['Sintesis'])
        print('Reseña: ')
        list_resenas = resenas(libro['Url'])
        for rs in list_resenas:
          print(rs)
        print('-----------------------------------')


def recomendaciones_por_genero():
  print('Géneros disponibles: ')
  for i in range(0, len(lista_generos), 3):
      print(f'{lista_generos[i]:<20}', end='')
      if i+1 < len(lista_generos):
          print(f'{lista_generos[i+1]:<20}', end='')
      if i+2 < len(lista_generos):
          print(f'{lista_generos[i+2]:<20}')
  print()
  seleccion = input('seleccione el genero deseado: ').lower()
  mejor_similitud = -1
  mejor_genero = None
  for genero in lista_generos:
      similitud = levenshtein_similarity(seleccion, genero)
      if similitud > mejor_similitud:
          mejor_similitud = similitud
          mejor_genero = genero
  df_genero = df.loc[df['Genero'] == mejor_genero]
  pregunta = [mejor_genero]
  modelo_genero = SentenceTransformer('msmarco-MiniLM-L-6-v3')
  incrustacion_consulta = modelo_genero.encode(pregunta)
  incrustaciones_respuestas = modelo_genero.encode(df_genero['Sintesis'].tolist())
  hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=3)
  print("Recomendados: ")
  for hit in hits[0]:
      indice_respuesta = hit['corpus_id']
      respuesta_similar = df_genero.iloc[indice_respuesta]
      print('Título: ', respuesta_similar['Titulo'])
      print('Autor: ', respuesta_similar['Autor'])
      print('Género: ', respuesta_similar['Genero'])
      print('Sintesis: ', respuesta_similar['Sintesis'])
      print('Reseña: ')
      list_resenas = resenas(respuesta_similar['Url'])
      for rs in list_resenas:
        print(rs)
      print('-----------------------------------')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Bot por linea de comando

In [87]:
print("Bienvenido al asistente de recomendación de libros!")
while True:
    print("¿Qué deseas hacer?")
    print("1. Recomendar un libro directamente")
    print("2. Buscar libros de un autor específico")
    print("3. Buscar libros de un género específico")
    print("4. Salir")
    opcion = input("Ingresa una opción: ")
    if opcion == "1":
        recomendacion_directa()
    elif opcion == "2":
        recomendacion_autor()
    elif opcion == "3":
        recomendaciones_por_genero()
    elif opcion == "4":
        print("Hasta pronto!")
        break
    else:
        print("Opción inválida. Inténtalo de nuevo.")

Bienvenido al asistente de recomendación de libros!
¿Qué deseas hacer?
1. Recomendar un libro directamente
2. Buscar libros de un autor específico
3. Buscar libros de un género específico
4. Salir
Ingresa una opción: 1
Ingrese la temática deseada: futbol
El género predicho es: deporte


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Recomendados: 
Título:  Nunca fuimos más felices
Autor:  carlos marzal
Género:  deporte
Sintesis:  Este libro es un tratado literario de filosofía epicúrea que reivindica la felicidad y ensalza el amor, la amistad, los libros, la bondad... a partir del fútbol. El narrador parte de los entrenamientos y partidos de su hijo, pero también de la pasión hacia las grandes ligas, los jugadores míticos o sus recuerdos infantiles como aficionado, con la «liturgia» que significaba ir al campo, y en cada uno de estos aspectos, en cada anécdota, en cada rememoración, abandona el fútbol para adentrarse en temas universales. Carlos Marzal va y viene, cuenta ritos de paso, escenas divertidísimas con padres que animan a sus hijos, conversaciones con otros escritores aficionados, historias sin fin que culminan en un capítulo emocionante, el del homenaje a su amigo Antonio Cabrera. El lector descubre entonces que el libro es en realidad un recorrido por la vida, y una invitación a lo mejor de ella, porqu

### Bot por telegram

Para poder utilizar el chatbot de telegram es necesario tener ejecutado el siguiente bloque de código y enviar un mensaje a: @TuiaNBBot

In [88]:
import logging
import telebot
from telebot import types
from sentence_transformers import SentenceTransformer, util
import Levenshtein

df = pd.read_csv('libros.csv')

TOKEN = '6977913405:AAFRzBHG-tmH5To5qw5iBOAwU-Qhafx0mng'
bot = telebot.TeleBot(TOKEN)
logging.basicConfig(filename='bot.log', level=logging.ERROR,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def create_category_keyboard():
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    recommend_button = types.KeyboardButton("Recomendar un libro")
    author_button = types.KeyboardButton("Buscar libros de un autor")
    genre_button = types.KeyboardButton("Buscar libros de un género")
    restart_button = types.KeyboardButton("Reiniciar")
    keyboard.row(recommend_button)
    keyboard.row(author_button, genre_button)
    keyboard.row(restart_button)
    return keyboard

def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    return 1 - (distance / max_len)

def recomendacion_directa(message):
    bot.send_message(message.chat.id, 'Ingrese la temática deseada:')
    bot.register_next_step_handler(message, procesar_tematica)

def procesar_tematica(message):
    tematica = message.text.lower()
    prediccion = predecir_genero_sinopsis(tematica, modelo_mlp)
    bot.send_message(message.chat.id, f"El género predicho es: {prediccion}")
    df_genero = df.loc[df['Genero'] == prediccion]
    pregunta = [tematica]
    modelo_genero = SentenceTransformer('msmarco-MiniLM-L-6-v3')
    incrustacion_consulta = modelo_genero.encode(pregunta)
    incrustaciones_respuestas = modelo_genero.encode(df_genero['Sintesis'].tolist())
    hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=3)
    bot.send_message(message.chat.id, "Recomendados:")
    for hit in hits[0]:
        indice_respuesta = hit['corpus_id']
        respuesta_similar = df_genero.iloc[indice_respuesta]
        resenas_libro = resenas(respuesta_similar['Url'])
        respuesta = (f"Título: {respuesta_similar['Titulo']}\n"
                     f"Autor: {respuesta_similar['Autor']}\n"
                     f"Género: {respuesta_similar['Genero']}\n"
                     f"Sintesis: {respuesta_similar['Sintesis']}\n"
                     "Reseñas:\n")
        for rs in resenas_libro:
            respuesta += f"- {rs}\n"
        respuesta += "-----------------------------------"
        bot.send_message(message.chat.id, respuesta)

modelo_autor = SentenceTransformer('msmarco-MiniLM-L-6-v3')

def recomendacion_autor(message):
    autores = df['Autor'].unique()
    autores_lista = [autor.lower() for autor in autores]
    autores_texto = '\n'.join([f'{autor}\n' for autor in autores_lista])
    bot.send_message(message.chat.id, 'Autores disponibles:\n' + autores_texto)
    bot.send_message(message.chat.id, 'Ingrese el autor que desea:')
    bot.register_next_step_handler(message, procesar_autor)


def procesar_autor(message):
    consulta_autor = message.text.lower()
    incrustacion_consulta = modelo_autor.encode([consulta_autor])
    incrustaciones_respuestas = modelo_autor.encode([autor.lower() for autor in df['Autor'].unique()])
    hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=6)
    subcadenas = [df['Autor'].unique()[hit['corpus_id']] for hit in hits[0] if consulta_autor in df['Autor'].unique()[hit['corpus_id']].lower()]
    recomendaciones = []

    def agregar_libros(autor, max_libros):
        libros_autor = df[df['Autor'].str.lower() == autor.lower()].head(max_libros)
        for _, libro in libros_autor.iterrows():
            recomendaciones.append(libro)
        return len(libros_autor)

    if not subcadenas:
        mejor_autor = df['Autor'].unique()[hits[0][0]['corpus_id']]
        agregar_libros(mejor_autor, 3)
    else:
        for autor in subcadenas:
            if len(recomendaciones) >= 3:
                break
            agregar_libros(autor, 3 - len(recomendaciones))

    if len(recomendaciones) < 3:
        for hit in hits[0]:
            autor_similar = df['Autor'].unique()[hit['corpus_id']]
            if autor_similar.lower() not in [autor.lower() for autor in subcadenas]:
                agregar_libros(autor_similar, 3 - len(recomendaciones))
                if len(recomendaciones) >= 3:
                    break

    for libro in recomendaciones:
        resenas_libro = resenas(libro['Url'])
        respuesta = (f"Título: {libro['Titulo']}\n"
                     f"Autor: {libro['Autor']}\n"
                     f"Género: {libro['Genero']}\n"
                     f"Sintesis: {libro['Sintesis']}\n"
                     "Reseñas:\n")
        for rs in resenas_libro:
            respuesta += f"- {rs}\n"
        respuesta += "-----------------------------------"
        bot.send_message(message.chat.id, respuesta)

def recomendaciones_por_genero(message):
    generos_texto = '\n'.join([f'{lista_generos[i]:<20}' + (f'{lista_generos[i+1]:<20}' if i+1 < len(lista_generos) else '') + (f'{lista_generos[i+2]:<20}' if i+2 < len(lista_generos) else '') for i in range(0, len(lista_generos), 3)])
    bot.send_message(message.chat.id, 'Géneros disponibles:\n' + generos_texto)
    bot.send_message(message.chat.id, 'Seleccione el género deseado:')
    bot.register_next_step_handler(message, procesar_genero)

def procesar_genero(message):
    seleccion = message.text.lower()
    mejor_similitud = -1
    mejor_genero = None
    for genero in lista_generos:
        similitud = levenshtein_similarity(seleccion, genero)
        if similitud > mejor_similitud:
            mejor_similitud = similitud
            mejor_genero = genero
    df_genero = df.loc[df['Genero'] == mejor_genero]
    pregunta = [mejor_genero]
    modelo_genero = SentenceTransformer('msmarco-MiniLM-L-6-v3')
    incrustacion_consulta = modelo_genero.encode(pregunta)
    incrustaciones_respuestas = modelo_genero.encode(df_genero['Sintesis'].tolist())
    hits = util.semantic_search(incrustacion_consulta, incrustaciones_respuestas, top_k=3)
    bot.send_message(message.chat.id, "Recomendados:")
    for hit in hits[0]:
        indice_respuesta = hit['corpus_id']
        respuesta_similar = df_genero.iloc[indice_respuesta]
        resenas_libro = resenas(respuesta_similar['Url'])
        respuesta = (f"Título: {respuesta_similar['Titulo']}\n"
                     f"Autor: {respuesta_similar['Autor']}\n"
                     f"Género: {respuesta_similar['Genero']}\n"
                     f"Sintesis: {respuesta_similar['Sintesis']}\n"
                     "Reseñas:\n")
        for rs in resenas_libro:
            respuesta += f"- {rs}\n"
        respuesta += "-----------------------------------"
        bot.send_message(message.chat.id, respuesta)


@bot.message_handler(commands=['start'])
def handle_start(message):
    try:
        bot.send_message(message.chat.id, "¡Hola! Soy un bot de recomendaciones de libros. ¡Bienvenido!\n"
                                          "¿Qué deseas hacer?\n"
                                          "1. Recomendar un libro directamente\n"
                                          "2. Buscar libros de un autor específico\n"
                                          "3. Buscar libros de un género específico\n"
                                          "4. Salir",
                         reply_markup=create_category_keyboard())
    except Exception as e:
        logger.error("Error en handle_start: %s", str(e))

@bot.message_handler(func=lambda message: message.text == "Recomendar un libro")
def handle_recommend_book(message):
    recomendacion_directa(message)

@bot.message_handler(func=lambda message: message.text == "Buscar libros de un autor")
def handle_search_author(message):
    recomendacion_autor(message)

@bot.message_handler(func=lambda message: message.text == "Buscar libros de un género")
def handle_search_genre(message):
    recomendaciones_por_genero(message)

@bot.message_handler(func=lambda message: message.text == "Reiniciar")
def handle_restart(message):
    bot.send_message(message.chat.id, "La conversación ha sido reiniciada.",
                     reply_markup=create_category_keyboard())

@bot.message_handler(func=lambda message: True)
def handle_text_messages(message):
    bot.reply_to(message, "Lo siento, solo puedes interactuar a través de los botones disponibles.")

if __name__ == '__main__':
    bot.polling()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Este trabajo práctico ha sido una oportunidad para integrar y aplicar diversas técnicas y herramientas del procesamiento de lenguaje natural, ampliando nuestra comprensión y capacidades en estos campos. La transición de la regresión a modelos más complejos y adecuados ha resaltado la importancia de la flexibilidad y la innovación en la resolución de problemas reales.